<a href="https://colab.research.google.com/github/JaroslavHolecek/Teaching/blob/master/JupyterNotebook/SQL/Vlastnosti_atributu_zadani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vlastnosti atributů (Column Constraints)

V předchozím notebooku jsme poznali příkazy `CREATE TABLE`, `ALTER TABLE` a `DROP TABLE`. Nyní se zaměříme na **vlastnosti (omezení)**, které můžeme jednotlivým sloupcům nastavit.

## Přehled vlastností

| Vlastnost | Popis |
|---|---|
| `PRIMARY KEY` | Jednoznačná identifikace řádku – hodnota se nesmí opakovat a nesmí být `NULL` |
| `AUTO_INCREMENT` | Automaticky doplní hodnotu (předchozí maximum + 1) |
| `NOT NULL` | Sloupec nesmí obsahovat prázdnou hodnotu (`NULL`) |
| `UNIQUE` | Hodnota ve sloupci se nesmí opakovat (ale může být `NULL`) |
| `DEFAULT` | Nastaví výchozí hodnotu, pokud ji při vkládání neuvedeme |
| `FOREIGN KEY` | Odkaz na primární klíč jiné tabulky – zajišťuje referenční integritu |
| `CONSTRAINT` | Pojmenované omezení – lze aplikovat i na kombinaci více sloupců |

---

# PRIMARY KEY (primární klíč)

Primární klíč slouží k **jednoznačné identifikaci** každého řádku v tabulce.

- Ve sloupci označeném jako `PRIMARY KEY` se **nesmí žádná hodnota opakovat** a nesmí být `NULL`.
- Databáze toto hlídá automaticky.
- Při dotazu podle primárního klíče dostaneme vždy **nejvýše jeden řádek**.
- Jako primární klíč téměř vždy používáme sloupec `id` spolu s vlastností `AUTO_INCREMENT`.

## Ukázka – jeden primární klíč

In [ ]:
!pip install mysql-connector-python

In [ ]:
from pripojeni import *
import mysql.connector

mydb = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)
mycursor = mydb.cursor()

mycursor.execute("""
CREATE TABLE IF NOT EXISTS automobily (
    id INT PRIMARY KEY AUTO_INCREMENT,
    jmeno VARCHAR(50),
    prijmeni VARCHAR(50)
)""")

mydb.commit()
mycursor.close()
mydb.close()

## Primární klíč přes více sloupců (složený klíč)

Primární klíč můžeme nastavit i přes **kombinaci více sloupců**. V tabulce se potom nesmí opakovat řádek se **stejnou kombinací** těchto hodnot (jednotlivé hodnoty se opakovat mohou).

K tomu použijeme `CONSTRAINT` (pojmenované omezení):

```sql
CONSTRAINT nazev_omezeni PRIMARY KEY (sloupec1, sloupec2)
```

> **Pojmenování omezení** se zobrazí např. v chybových hláškách – díky tomu snáze najdeme problém.

In [ ]:
from pripojeni import *
import mysql.connector

mydb = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)
mycursor = mydb.cursor()

# Smažeme tabulku z předchozího příkladu
mycursor.execute("DROP TABLE IF EXISTS automobily")

mycursor.execute("""
CREATE TABLE automobily (
    id INT,
    jmeno CHAR(10),
    prijmeni CHAR(20),
    CONSTRAINT pk_jmeno_prijmeni PRIMARY KEY (jmeno, prijmeni)
)""")

mydb.commit()
mycursor.close()
mydb.close()

# FOREIGN KEY (cizí klíč)

Pomocí cizího klíče říkáme databázi, že hodnoty v daném sloupci **odkazují na primární klíč jiné tabulky**. Databáze pak hlídá **referenční integritu**:

- Nelze vložit hodnotu, která v odkazované tabulce neexistuje.
- Nelze smazat řádek z odkazované tabulky, pokud na něj někdo odkazuje (pokud nenastavíme jinak).

### Chování při mazání odkazovaného řádku

| Volba | Chování |
|---|---|
| *(výchozí)* | Smazání řádku s odkazem **selže s chybou** |
| `ON DELETE CASCADE` | Automaticky smaže i všechny odkazující řádky |
| `ON DELETE SET NULL` | Nastaví cizí klíč na `NULL` |

> **Důležité:** Odkazovaný sloupec musí být `PRIMARY KEY` nebo `UNIQUE` – aby byla odkazovaná hodnota vždy jednoznačná.

In [ ]:
from pripojeni import *
import mysql.connector

mydb = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)
mycursor = mydb.cursor()

mycursor.execute("DROP TABLE IF EXISTS osoby")
mycursor.execute("DROP TABLE IF EXISTS kvalifikace")

mycursor.execute("""
CREATE TABLE kvalifikace (
    id INT PRIMARY KEY AUTO_INCREMENT,
    oznaceni VARCHAR(50),
    popis TEXT
)""")
mydb.commit()

mycursor.execute("""
CREATE TABLE osoby (
    id INT PRIMARY KEY AUTO_INCREMENT,
    jmeno VARCHAR(50),
    prijmeni VARCHAR(50),
    datum_narozeni DATE,
    kvalifikace_id INT,
    FOREIGN KEY (kvalifikace_id) REFERENCES kvalifikace(id)
)""")
mydb.commit()

mycursor.close()
mydb.close()

Sloupec `kvalifikace_id` jsme nastavili jako cizí klíč odkazující na sloupec `id` v tabulce `kvalifikace`. Díky tomu můžeme do `kvalifikace_id` zapsat pouze hodnotu, která existuje v `kvalifikace.id`.

> **Konvence:** Sloupec s cizím klíčem pojmenováváme ve tvaru `nazev_odkazovane_tabulky_id` — např. `kvalifikace_id`, `auto_id`.

---

# NOT NULL

Sloupec s vlastností `NOT NULL` **nesmí obsahovat prázdnou hodnotu** (`NULL`). Při každém vkládání řádku musíme tuto hodnotu vyplnit.

```sql
jmeno VARCHAR(50) NOT NULL
```

---

# AUTO_INCREMENT

Databáze automaticky doplní hodnotu — zjistí nejvyšší existující hodnotu ve sloupci, přičte 1 a výsledek vloží. Typicky se kombinuje s `PRIMARY KEY` na sloupci `id`.

```sql
id INT PRIMARY KEY AUTO_INCREMENT
```

---

# UNIQUE

Ve sloupci s touto vlastností se **nesmí žádná hodnota opakovat** (na rozdíl od `PRIMARY KEY` může obsahovat `NULL`).

Lze nastavit i na více sloupců — potom se nesmí opakovat **konkrétní kombinace** hodnot, jednotlivé hodnoty se opakovat mohou.

```sql
-- Na jednom sloupci
spz CHAR(7) UNIQUE

-- Na kombinaci sloupců (pomocí CONSTRAINT)
CONSTRAINT unikatni_jmeno UNIQUE (jmeno, prijmeni)
```

---

# DEFAULT

Nastaví výchozí hodnotu sloupce. Pokud při vkládání hodnotu neuvedeme, automaticky se doplní tato výchozí.

```sql
stav VARCHAR(20) DEFAULT 'aktivni'
```

---

# CONSTRAINT (pojmenované omezení)

`CONSTRAINT` používáme, když chceme:

- **Pojmenovat omezení** — název se zobrazí v chybových hláškách, což usnadní ladění.
- **Nastavit omezení přes více sloupců** — např. kombinace `(jmeno, prijmeni)` musí být unikátní.
- **Přidat omezení dodatečně** pomocí `ALTER TABLE`.

## Zápis přímo při vytváření tabulky

In [ ]:
from pripojeni import *
import mysql.connector

mydb = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)
mycursor = mydb.cursor()

mycursor.execute("DROP TABLE IF EXISTS osoby")
mycursor.execute("DROP TABLE IF EXISTS kvalifikace")

mycursor.execute("""
CREATE TABLE osoby (
    jmeno CHAR(10) NOT NULL,
    prijmeni CHAR(20) NOT NULL,
    datum_narozeni DATE,
    kvalifikace VARCHAR(50),
    CONSTRAINT pk_osoby PRIMARY KEY (jmeno, prijmeni),
    CONSTRAINT unikatni_narozeni UNIQUE (datum_narozeni)
)""")

mydb.commit()
mycursor.close()
mydb.close()

## Zápis omezení dodatečně (ALTER TABLE)

In [ ]:
from pripojeni import *
import mysql.connector

mydb = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)
mycursor = mydb.cursor()

mycursor.execute("DROP TABLE IF EXISTS osoby")

mycursor.execute("""
CREATE TABLE osoby (
    jmeno CHAR(10) NOT NULL,
    prijmeni CHAR(20) NOT NULL,
    datum_narozeni DATE,
    kvalifikace VARCHAR(50)
)""")
mydb.commit()

# Přidání primárního klíče dodatečně
mycursor.execute("""
    ALTER TABLE osoby
    ADD CONSTRAINT pk_osoby PRIMARY KEY (jmeno, prijmeni)
""")
mydb.commit()

# Přidání UNIQUE omezení dodatečně
mycursor.execute("""
    ALTER TABLE osoby
    ADD CONSTRAINT unikatni_narozeni UNIQUE (datum_narozeni)
""")
mydb.commit()

mycursor.close()
mydb.close()

---

# Cvičení 1

Připojte se k databázi a vytvořte dvě tabulky:

### Tabulka `auta`
- `id` — INT, PRIMARY KEY, AUTO_INCREMENT
- `znacka` — CHAR(20), NOT NULL
- `spz` — CHAR(7), NOT NULL, UNIQUE

### Tabulka `ridici`
- `id` — INT, PRIMARY KEY, AUTO_INCREMENT
- `jmeno` — CHAR(10), NOT NULL
- `prijmeni` — CHAR(20), NOT NULL
- `auto_id` — INT, FOREIGN KEY odkazující na `auta(id)`
- Kombinace `jmeno` a `prijmeni` musí být unikátní → `CONSTRAINT unikatni_jmeno`

> **Pozor na pořadí:** Nejdříve vytvořte tabulku `auta` (na ni se odkazuje cizí klíč), pak teprve `ridici`.

<details>
<summary>Očekávaný výstup</summary>

```
Tabulka auta vytvořena správně.
Tabulka ridici vytvořena správně.
```

</details>

In [ ]:
from pripojeni import *
import mysql.connector

mydb = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)
mycursor = mydb.cursor()

# === ZDE PŘIJDE VÁŠ KÓD ===


# === KONEC VAŠEHO KÓDU (tuto část neměňte!) ===

mycursor.execute("DESCRIBE auta")
table = str(mycursor.fetchall())
# Normalizace pro kompatibilitu s různými verzemi MySQL (5.x vs 8.0+)
table = table.replace("int(11)", "int")

expected = "[('id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('znacka', 'char(20)', 'NO', '', None, ''), ('spz', 'char(7)', 'NO', 'UNI', None, '')]"
if table == expected:
    print("Tabulka auta vytvořena správně.")
else:
    print("Tabulka auta není dle zadání.\nVaše tabulka:")
    print(table)
    print("Očekávaná tabulka:")
    print(expected)
    print("Výpis se liší na pozicích:", [i for i in range(min(len(table), len(expected))) if table[i] != expected[i]])


mycursor.execute("DESCRIBE ridici")
table = str(mycursor.fetchall())
table = table.replace("int(11)", "int")

expected = "[('id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('jmeno', 'char(10)', 'NO', '', None, ''), ('prijmeni', 'char(20)', 'NO', '', None, ''), ('auto_id', 'int', 'YES', 'MUL', None, '')]"
if table == expected:
    print("Tabulka ridici vytvořena správně.")
else:
    print("Tabulka ridici není dle zadání.\nVaše tabulka:")
    print(table)
    print("Očekávaná tabulka:")
    print(expected)
    print("Výpis se liší na pozicích:", [i for i in range(min(len(table), len(expected))) if table[i] != expected[i]])

# Úklid – smažeme tabulky (ridici první kvůli FK)
mycursor.execute("DROP TABLE ridici")
mycursor.execute("DROP TABLE auta")
mydb.commit()

mycursor.close()
mydb.close()